# Processing of Biological Data
**Prof. Dr. Volkhard Helms**
**Tutor:** Hanah Robertson
**Winter Semester 2025–26**
**Saarland University – Chair for Computational Biology**

## Exercise Sheet 2
**Due:** 19.11.2025, 10:00 am

**Student:** Ramin Yazdani
**Matriculation No.:** 7068679

**Student:** Kamyar Kashefzand
**Matriculation No.:** 7062527


## Imports / setup


In [17]:
import math
import numpy as np
import pandas as pd


## Step 1 – Loading and inspecting the methylation data

In this step, We first load the `methylation.csv` file and separate:

- **Metadata columns** (chromosome position, gene annotation, etc.).
- **Cell-type columns** (DNA methylation values for each cell type).

The file uses:

- `;` as separator,
- `,` as decimal separator,
- `.` to indicate missing values.

Therefore, We have to explicitly configure the CSV reader.


In [18]:
# Load the methylation CSV with correct parsing settings
df = pd.read_csv(
    "../data/methylation.csv",
    sep=";",          # semicolon-separated
    decimal=",",      # comma as decimal separator
    na_values=".",    # "." represents missing values
    engine="python"   # more flexible parser for irregular CSVs
)

# Known metadata columns from the file
metadata_cols = [
    "chrom",
    "chromStart",
    "chromEnd",
    "name",
    "geneName",
    "ensemblId",
    "cpgMinCoverage",
]

# Finding cell-types
cell_types = [c for c in df.columns if c not in metadata_cols]

# Extract methylation matrix (only cell-type columns)
meth = df[cell_types]

# Drop rows that contain any NA in methylation columns
meth_clean = meth.fillna(0)

df.shape, meth_clean.shape, metadata_cols, cell_types[:10]



((95086, 26),
 (95086, 19),
 ['chrom',
  'chromStart',
  'chromEnd',
  'name',
  'geneName',
  'ensemblId',
  'cpgMinCoverage'],
 ['HSC', 'MPP1', 'MPP2', 'CLP', 'CMP', 'GMP', 'MEP', 'CD4', 'CD8', 'B_cell'])

### Explanation – Data loading and preprocessing

To distinguish metadata from methylation data:

- We define the metadata column names explicitly:  
  `chrom`, `chromStart`, `chromEnd`, `name`, `geneName`, `ensemblId`, `cpgMinCoverage`.
- All other columns are cell-type methylation columns, which we store in `cell_types`.
- We create the methylation matrix `meth` and updated rows with missing methylation values to obtain `meth_clean`.

`meth_clean` is the matrix used for downstream analysis: rows = genomic regions, columns = cell types


In [19]:
# Average methylation level per cell type
mean_per_cell = meth_clean.mean()

mean_per_cell


HSC       0.596141
MPP1      0.617036
MPP2      0.628850
CLP       0.640219
CMP       0.641742
GMP       0.635085
MEP       0.607004
CD4       0.646955
CD8       0.648090
B_cell    0.646083
Eryth     0.516473
Granu     0.621114
Mono      0.625927
TBSC      0.568291
ABSC      0.569460
MTAC      0.538288
CLDC      0.546543
EPro      0.585096
EDif      0.582280
dtype: float64

## Step 2 – Methylation levels across cell types

The mean methylation values per cell type are roughly in the range 0.52–0.65. For example:

- Blood-related cell types like **CD4**, **CD8**, and **B_cell** show mean methylation around ~0.65.
- Progenitor and myeloid cells (**HSC**, **MPP1**, **MPP2**, **CLP**, **CMP**, **GMP**, **MEP**, **Granu**, **Mono**) are mostly between ~0.60 and ~0.64.
- The erythroid lineage (**Eryth**) has a somewhat lower mean (~0.52).
- Skin-related cell types (**TBSC**, **ABSC**, **MTAC**, **CLDC**, **EPro**, **EDif**) are in an intermediate range (~0.54–0.59).

Overall, most cell types have moderate to high methylation on average, which fits the general expectation that large parts of the genome (especially intergenic and repetitive regions) are methylated, while only specific regulatory regions (e.g. promoters of active genes) are hypomethylated.


In [20]:
# Compare methylation in genic vs non-genic regions using 'geneName' as proxy

if "geneName" in df.columns:
    # Align geneName info with rows that survived dropna in meth_clean
    has_gene = df.loc[meth_clean.index, "geneName"].notna()

    mean_genic = meth_clean[has_gene].mean().mean()
    mean_nongenic = meth_clean[~has_gene].mean().mean()

    print("Mean methylation in genic regions:    ", mean_genic)
    print("Mean methylation in non-genic regions:", mean_nongenic)
else:
    print("Column 'geneName' not found.")



Mean methylation in genic regions:     0.5507375166245346
Mean methylation in non-genic regions: 0.7417760965668222


### Developmental succession and methylation patterns

It is known that methylation generally **increases with specification** during development.  
To check this, I compare the mean methylation values of the blood lineage in developmental order:

- HSC: **0.596**
- MPP1: **0.617**
- MPP2: **0.628**
- CLP: **0.640**
- CMP: **0.642**
- GMP: **0.635**
- MEP: **0.607**
- CD4: **0.647**
- CD8: **0.648**
- B_cell: **0.646**
- Granu: **0.621**
- Mono: **0.625**

**Interpretation**

- From **HSC → MPP1 → MPP2 → CLP/CMP**, methylation **does increase**, consistent with the expectation that differentiation leads to more lineage-locked methylation.
- However, not all later branches strictly increase:
  - **MEP** has lower average methylation (~0.607) compared to CLP/CMP.  
    This is reasonable, because erythroid/megakaryocytic chromatin becomes globally more accessible during terminal maturation.
  - Mature immune cells (CD4, CD8, B cells) again show **higher methylation** (~0.646–0.648).

Thus, the data show **partial agreement** with the expectation “methylation increases with specification”:
- The early progenitor stages match the expected trend.
- Later differentiation branches deviate depending on lineage-specific chromatin remodeling.

### Expected patterns in genomic regions

The observed genic vs non-genic values:
- Genic regions: **0.55**
- Non-genic regions: **0.74**

match classical epigenetic principles:
- **Genic regions**, especially promoters and active regulatory elements, tend to be hypomethylated.
- **Intergenic regions** and repeats are generally hypermethylated.

These patterns hold across all differentiation stages in this dataset.



## Step 3 – Euclidean distance between cell types

Now we have to compute Euclidean distance to quantify how similar or different two cell types are in terms of their methylation profiles.

In the next cell, we implement this as a function and compute distances between the three cell types
requested in the exercise: HSC, CD8, and MTAC.


In [21]:
def euclidean_distance(col_a: str, col_b: str, data: pd.DataFrame) -> float:
    """
    Compute Euclidean distance between two cell types (columns) over all regions (rows).
    """
    a = data[col_a].to_numpy()
    b = data[col_b].to_numpy()
    diff = a - b
    return float(np.sqrt(np.sum(diff ** 2)))

# Compute distances between HSC, CD8, and MTAC (if present)
for a, b in [("HSC", "CD8"), ("HSC", "MTAC"), ("CD8", "MTAC")]:
    if a in cell_types and b in cell_types:
        d = euclidean_distance(a, b, meth_clean)
        print(f"d({a}, {b}) = {d:.2f}")
    else:
        print(f"One of {a} or {b} is not found among cell types.")


d(HSC, CD8) = 46.06
d(HSC, MTAC) = 78.55
d(CD8, MTAC) = 81.78


### Interpretation of Euclidean distances


A **smaller distance** means the two cell types have more similar genome-wide methylation patterns.

From these values we can conclude:

- **HSC** and **CD8** are much closer to each other than either is to **MTAC**. This is expected, since both HSC and CD8 belong to the **hematopoietic (blood) lineage**.
- Distances involving **MTAC** are substantially higher, reflecting that MTAC is a **skin-associated** cell type, with a methylation profile that is quite different from the blood cells.

These distances already hint at a strong separation between blood and skin lineages in methylation space, which is then further explored with hierarchical clustering.


## Step 4 – Average linkage between clusters

Hierarchical clustering merges **clusters** of cell types step by step. To decide which two clusters to merge next, we implemented average linkage:

Before I can compute average linkage, we pre-compute all pairwise Euclidean distances between cell types and store them in a dictionary.


In [22]:
# Precompute all pairwise distances between cell types
pairwise_dist = {}
n = len(cell_types)

for i in range(n):
    for j in range(i + 1, n):
        pairwise_dist[(i, j)] = euclidean_distance(cell_types[i], cell_types[j], meth_clean)

# Just to sanity-check a few distances:
print("Number of cell types:", n)
print("Number of stored pairwise distances:", len(pairwise_dist))

# Define average linkage between two clusters (lists of indices)
def avg_linkage(cluster_A, cluster_B):
    """
    Compute average linkage between two clusters of indices.
    cluster_A, cluster_B: lists of indices into cell_types
    """
    total = 0.0
    count = 0

    for i in cluster_A:
        for j in cluster_B:
            if i == j:
                continue
            a, b = sorted((i, j))
            total += pairwise_dist[(a, b)]
            count += 1

    return total / count



Number of cell types: 19
Number of stored pairwise distances: 171


## Step 5 – Agglomerative hierarchical clustering (average linkage)

Now it is time to implement **agglomerative hierarchical clustering**:

1. Start with each cell type as its own cluster.
2. At each iteration:
   - Compute the average linkage between all pairs of active clusters.
   - Merge the pair of clusters with the smallest average linkage.
3. Repeat until only one cluster remains.

we store a `history` list that records, for each merge:
- which clusters were merged,
- at which linkage (distance) level,
- and which members the new cluster contains.



In [23]:
# Initialize clusters: each cell type is its own cluster (by index)
clusters = {i: [i] for i in range(n)}
active = set(clusters.keys())
next_id = n  # id for newly formed clusters

history = []

while len(active) > 1:
    best_pair = None
    best_link = float("inf")
    active_list = list(active)

    # Find the pair of clusters with minimal average linkage
    for i_idx in range(len(active_list)):
        for j_idx in range(i_idx + 1, len(active_list)):
            ci = active_list[i_idx]
            cj = active_list[j_idx]
            link = avg_linkage(clusters[ci], clusters[cj])
            if link < best_link:
                best_link = link
                best_pair = (ci, cj)

    ci, cj = best_pair
    membersA = clusters[ci]
    membersB = clusters[cj]
    new_members = membersA + membersB

    # Record the merge step
    history.append({
        "step": len(history) + 1,
        "clusterA": [cell_types[i] for i in membersA],
        "clusterB": [cell_types[i] for i in membersB],
        "linkage": best_link,
        "merged": [cell_types[i] for i in new_members],
    })

    # Merge the clusters
    del clusters[ci]
    del clusters[cj]
    active.remove(ci)
    active.remove(cj)

    clusters[next_id] = new_members
    active.add(next_id)
    next_id += 1


In [24]:
# Show the first 15 merge steps to understand the clustering structure
for h in history:
    print(
        f"Step {h['step']:2d}:",
        h["clusterA"], "+", h["clusterB"],
        "->", h["merged"],
        "@ linkage =", round(h["linkage"], 2)
    )


Step  1: ['MPP2'] + ['CMP'] -> ['MPP2', 'CMP'] @ linkage = 15.64
Step  2: ['CD4'] + ['CD8'] -> ['CD4', 'CD8'] @ linkage = 16.09
Step  3: ['GMP'] + ['MPP2', 'CMP'] -> ['GMP', 'MPP2', 'CMP'] @ linkage = 19.6
Step  4: ['CLP'] + ['GMP', 'MPP2', 'CMP'] -> ['CLP', 'GMP', 'MPP2', 'CMP'] @ linkage = 22.21
Step  5: ['B_cell'] + ['CLP', 'GMP', 'MPP2', 'CMP'] -> ['B_cell', 'CLP', 'GMP', 'MPP2', 'CMP'] @ linkage = 23.98
Step  6: ['Granu'] + ['B_cell', 'CLP', 'GMP', 'MPP2', 'CMP'] -> ['Granu', 'B_cell', 'CLP', 'GMP', 'MPP2', 'CMP'] @ linkage = 25.24
Step  7: ['EPro'] + ['EDif'] -> ['EPro', 'EDif'] @ linkage = 25.62
Step  8: ['MEP'] + ['Granu', 'B_cell', 'CLP', 'GMP', 'MPP2', 'CMP'] -> ['MEP', 'Granu', 'B_cell', 'CLP', 'GMP', 'MPP2', 'CMP'] @ linkage = 26.45
Step  9: ['CD4', 'CD8'] + ['MEP', 'Granu', 'B_cell', 'CLP', 'GMP', 'MPP2', 'CMP'] -> ['CD4', 'CD8', 'MEP', 'Granu', 'B_cell', 'CLP', 'GMP', 'MPP2', 'CMP'] @ linkage = 28.35
Step 10: ['MPP1'] + ['CD4', 'CD8', 'MEP', 'Granu', 'B_cell', 'CLP', 'GMP

## Step 6 – Final Interpretation of the Complete Clustering (All 18 Steps)

After replacing missing values with 0 and re-running the full hierarchical clustering, the final merge history (18 steps) shows a clear and biologically meaningful structure separating **blood** and **skin** cell types, with cluster merges that partially reflect hematopoietic differentiation.



### **Blood lineage structure**

Across Steps 1–15, a large, coherent **blood cluster** progressively forms:

- The earliest merges combine hematopoietic progenitors:
  - **MPP2 + CMP**
  - then **GMP**, **CLP**, **B_cell**, **Granu**
- Later, differentiating and lineage-specific cells join:
  - **MEP**, then the **CD4/CD8** T-cell pair,
  - followed by **MPP1** (an upstream progenitor),
  - and **Mono** (myeloid immune cell).
- Finally, **HSC**, the stem cell at the top of the biological hierarchy, merges at a higher linkage distance.

This means the blood branch accumulates:

**{HSC, MPP1, MPP2, CMP, CLP, GMP, B_cell, Granu, Mono, CD4, CD8, MEP}**

before interacting with any skin-associated types.

This progressive merge pattern reflects several aspects of known hematopoiesis:
- Early multipotent progenitors cluster first.
- Myeloid and lymphoid derivatives accumulate later.
- HSC merges last within the blood lineage due to its globally distinct methylation profile.



### **Skin lineage structure**

The skin lineage behaves very cleanly:

- **EPro + EDif** form a differentiated epidermal pair.
- **TBSC + ABSC** form a stem/keratinocyte-like pair.
- **CLDC** joins TBSC/ABSC.
- In Step 14, the EPro/EDif and TBSC/ABSC/CLDC groups merge, forming a unified **skin cluster**.
- In Step 16, **MTAC** also merges into this skin lineage group.

Thus the skin cluster is:

**{EPro, EDif, TBSC, ABSC, CLDC, MTAC}**

This remains completely separate from the blood lineage until Step 17.



### **Final merges and lineage separation**

- **Step 17:**  
  The entire **blood cluster** merges with the entire **skin cluster**, showing that these two lineages are globally distinct and only join at a high linkage distance.

- **Step 18:**  
  **Eryth** (erythroid lineage) merges *last* into the combined cluster.  
  This agrees with its atypical, globally hypomethylated profile observed in earlier analyses.


![dendrogram](./dendogram.jpg)
### **Are the two lineages separable?**

**Yes.**  
Blood and skin cell types remain in completely separate branches through the first 16 merges, only joining at a very high linkage distance. This shows that DNA methylation strongly separates these two major lineages.



### **Can we see the developmental succession of blood cells?**

**Partially.**

- Early hematopoietic progenitors (MPP2, CMP, GMP, CLP) merge first.
- Then differentiated immune and myeloid subsets (B_cell, Granu, Mono).
- CD4/CD8 T cells merge as a pair reflecting their close functional similarity.
- HSC merges into the blood branch at a late stage, consistent with stem cells being epigenetically very distinct.

While the clustering does not perfectly reconstruct the biological hematopoiesis tree, it does capture major developmental relationships and a clear progression from progenitors → mature immune cells.



### **Overall conclusion**

The complete hierarchical clustering provides:
- A strong separation between **blood** and **skin** cell lineages.
- A meaningful internal structure within each lineage.
- A merge order that reflects parts of known biological differentiation (especially in hematopoiesis).

Thus, DNA methylation patterns alone are sufficient to recover major lineage identities and aspects of cellular developmental relationships.

